# Analyse des transactions immobilières en France  
*Une approche par les caractéristiques du logement, le contexte géographique et la proximité aux gares SNCF*

## 1. Données et préparation

### 1.1 Chargement et nettoyage initial

In [ ]:
# On effectue les importations basiques utiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Chemin vers le fichier DVF (données des transactions immobilières)
FILE_PATH = "/home/onyxia/work/Projet-Python-2A-ENSAE/ValeursFoncieres-2025-S1.csv"

In [ ]:
# --- 1ERE LECTURE DU FICHIER DVF ---

# On lit le fichier CSV avec pandas
df = pd.read_csv(
    FILE_PATH,       
    sep="|",        
    dtype=str,         
    decimal=",",       
    na_values=["", " "],  # les champs vides sont considérés comme des valeurs manquantes
    engine="python"    
)

print(df.head())

In [ ]:
# --- NETTOYAGE DE BASE DES VARIABLES ---

df["Date mutation"] = pd.to_datetime(
    df["Date mutation"],
    format="%d/%m/%Y",
    errors="coerce"    # les dates invalides deviennent NaT
)

df["Valeur fonciere"] = (
    df["Valeur fonciere"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

df["Surface terrain"] = pd.to_numeric(
    df["Surface terrain"],
    errors="coerce"
)

df["Surface reelle bati"] = pd.to_numeric(
    df["Surface reelle bati"],
    errors="coerce"
)

df["Nombre pieces principales"] = pd.to_numeric(
    df["Nombre pieces principales"],
    errors="coerce"
)


# --- OPTIONS D’AFFICHAGE (CONFORT DE LECTURE) ---

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_rows", 20)


# --- VÉRIFICATIONS DE BASE ---

print(df.head())
print(df.columns)
print(f"Nombre de lignes : {len(df)}")

In [ ]:
# On supprime les biens dont la surface bâtie, et la valeur foncière, est négative ou nulle
df = df[(df["Surface reelle bati"] > 0) & (df["Valeur fonciere"] > 0)]

print(f"Nombre de lignes : {len(df)}")

In [ ]:
# Liste des colonnes jugées non pertinentes pour l'analyse économique
colonnes_hors_perimetre = [
    "1 Articles CGI",
    "2 Articles CGI",
    "3 Articles CGI",
    "4 Articles CGI",
    "5 Articles CGI",
    "Identifiant de document",
    "Reference document"
]

df = df.drop(columns=colonnes_hors_perimetre, errors='ignore') # ignore pour quand on relance
df.head()

In [ ]:
# Comptage des types de locaux présents dans la base
df["Type local"].value_counts(dropna=False)

### 1.2 Définition du périmètre d’étude

La base DVF contient à la fois des biens résidentiels et des biens non résidentiels
(locaux industriels, commerciaux ou assimilés).

Dans ce projet, nous nous concentrons exclusivement sur le **marché du logement**.
Nous restreignons donc l’échantillon aux **maisons et appartements**, en nous appuyant
sur la variable *Type local*.

Ce filtrage permet d’éviter que des biens atypiques (entrepôts, locaux commerciaux,
bâtiments industriels) ne biaisent l’analyse des prix et des surfaces.


In [ ]:
# Filtrage des biens résidentiels : maisons et appartements uniquement
df = df[df["Type local"].isin(["Maison", "Appartement"])]

# Vérification après filtrage
df["Type local"].value_counts()

## 2. Variable cible et analyse descriptive

Dans ce projet, nous cherchons à expliquer et à prédire le niveau des prix immobiliers à partir des caractéristiques des biens et de leur environnement.

Nous avons choisi comme **variable cible** le **prix au mètre carré**, noté *Valeur m2*, plutôt que la valeur foncière totale.

Ce choix s’explique par plusieurs raisons :

- le prix au mètre carré permet de comparer des biens de tailles différentes sur une base homogène ;
- il neutralise l’effet mécanique de la surface sur le prix total ;
- il est largement utilisé dans les analyses immobilières (agents immobiliers, notaires, études économiques) ;
- il est plus pertinent pour analyser les variations spatiales et structurelles des prix.

Le prix au mètre carré est alors défini comme le rapport entre la valeur foncière et la surface réelle bâtie du bien.


In [ ]:
# --- CALCUL DU PRIX AU MÈTRE CARRÉ ---

df["Valeur m2"] = df["Valeur fonciere"] / df["Surface reelle bati"]

# --- VÉRIFICATION DE COHÉRENCE DU M2 ET DE LA SURFACE DU BÂTI ---

print(df["Valeur m2"].describe(),df["Surface reelle bati"].describe())

### 2.1 Choix du prix au mètre carré

L’analyse de la distribution du prix au mètre carré met en évidence la présence de valeurs extrêmes irréalistes, aussi bien très faibles que très élevées.

Nous remarquons le même problème avec la surface du bâti.

Ces observations correspondent à des anomalies de la base DVF ou à des biens atypiques.

Afin d’améliorer la lisibilité des graphiques et d’éviter que ces valeurs ne dominent l’analyse descriptive, nous appliquons un filtrage statistique basé sur les quantiles.

Concrètement, nous conservons uniquement les observations comprises entre les quantiles 5 % et 95 % du prix au mètre carré.  
Ce filtrage est utilisé uniquement pour l’exploration des données et n’affecte pas le jeu de données utilisé pour la modélisation.

In [ ]:
# Calcul des seuils 3 % et 97 % du prix au m²

pd.options.display.float_format = '{:.2f}'.format

df_eda = df

for X in ["Valeur m2", "Surface reelle bati"] :
    q_low = df_eda[X].quantile(0.03)
    q_high = df_eda[X].quantile(0.97)
    df_eda = df_eda[(df_eda[X] >= q_low) & (df_eda[X] <= q_high)]

# Vérification rapide
print(df_eda["Valeur m2"].describe(),df_eda["Surface reelle bati"].describe())

### 2.2 Distribution du prix au m²

Nous commençons par analyser la distribution du prix au mètre carré (*Valeur m2*) sur les données stabilisées pour l’analyse descriptive.

Cette étape permet de :
- vérifier les ordres de grandeur ;
- observer la dispersion des prix ;
- identifier d’éventuelles asymétries de la distribution.


In [ ]:
# Histogramme du prix au m²
df_eda["Valeur m2"].plot.hist(
    bins=50,
    edgecolor="black",
)

In [ ]:
# Boxplot du prix au m²
df_eda.boxplot(column="Valeur m2")

> La présence de nombreux outliers à la hausse reflète la forte hétérogénéité du marché immobilier, avec des biens situés dans des zones très tendues ou atypiques.


### 2.3 Effets structurels du logement

#### Effet taille : relation entre surface et prix au mètre carré

Nous étudions maintenant la relation entre la surface réelle bâtie et le prix au mètre carré.

En immobilier, il est courant d’observer un **effet taille** :
à mesure que la surface augmente, le prix au m² tend à diminuer.


In [ ]:
# Nuage de points : surface bâtie vs prix au m²
df_eda.plot.scatter(
    x="Surface reelle bati",
    y="Valeur m2",
    alpha=0.2
)


#### Comparaison maisons vs appartements

Nous comparons maintenant les prix au mètre carré entre maisons et appartements.
Ces deux types de biens obéissent à des logiques de marché différentes,
ce qui peut expliquer des niveaux de prix et des effets taille distincts.


In [ ]:
# Boxplot du prix au m² par type de bien
df_eda.boxplot(
    column="Valeur m2",
    by="Type local",
    grid=False
)

> Le boxplot montre une différence nette de niveau de prix entre appartements et maisons.
Les appartements présentent un prix au mètre carré médian plus élevé et une dispersion plus importante, ce qui reflète leur localisation plus fréquente en zones urbaines tendues.
Cette observation justifie l’intégration du type de bien comme variable explicative clé.

## 3. Modélisations sans localisation fine

### 3.1 Première régression linéaire

Nous estimons une régression linéaire afin d’expliquer le prix au mètre carré (*Valeur m2*)
à partir de caractéristiques intrinsèques du logement.

Variables explicatives retenues :
- la surface réelle bâtie (effet taille) ;
- le type de bien (maison ou appartement) ;
- le nombre de pièces principales.

Remarque : notre jeu de données ne couvre que l’année 2025, donc nous n’introduisons pas de variable temporelle.

In [ ]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

#### Préparation des données pour la régression

Nous créons une variable binaire pour représenter le type de bien :
- Appartement = 1
- Maison = 0

Nous sélectionnons ensuite les variables explicatives et la variable cible.
Enfin, nous supprimons les lignes avec valeurs manquantes pour éviter les erreurs lors de l’entraînement.

In [ ]:
df_model = df.copy()

# Encodage du type de bien : 1 si appartement, 0 si maison
df_model["Appartement"] = (df_model["Type local"] == "Appartement").astype(int)

df_model_clean = df_model[["Surface reelle bati", "Appartement", "Nombre pieces principales","Valeur m2"]].dropna() # Suppression des lignes contenant au moins une valeur manquante
print(df_model.shape[0]-df_model_clean.shape[0])

X = df_model_clean[["Surface reelle bati", "Appartement", "Nombre pieces principales"]]
y = df_model_clean["Valeur m2"]

In [ ]:
# Séparation train / test (80 % / 20 %), avec une graine fixe pour la reproductibilité
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

model = LinearRegression()

# Entraînement du modèle
model.fit(X_train1, y_train1)

# Prédictions sur les données de test
y_pred1 = model.predict(X_test1)

print(r2_score(y_test1, y_pred1))
print(mean_absolute_error(y_test1, y_pred1))


On remarque que le modèle *naïf* de régression linéaire par rapport aux propriétés intrinsèques du logement est peu pertinent.

Pour que le résultat soit plus visuel :

In [ ]:
coefficients = pd.Series(
    model.coef_,
    index=X.columns,
)
coefficients["Intercept"] = model.intercept_

print(coefficients)

### 3.2 Spécification logarithmique

In [ ]:
# Transformation logarithmique
X_log = X.copy()
X_log["Surface reelle bati"] = np.log(X_log["Surface reelle bati"])
y_log = np.log(y)

# On refait la même chose que tout à l'heure
X_train1_log, X_test1_log, y_train1_log, y_test1_log = train_test_split(
    X_log, y_log, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train1_log, y_train1_log)

print(r2_score(y_test1_log, model.predict(X_test1_log)))

> En passant à une spécification logarithmique, le pouvoir explicatif du modèle augmente nettement.
Le R² atteint environ 10 %, ce qui confirme que les caractéristiques intrinsèques du logement expliquent une partie non négligeable du prix au mètre carré.
La part restante s’explique principalement par la localisation, que nous introduirons dans un second temps.

## 4. Introduction de la localisation

### 4.1 Localisation grossière : département

Avant d’introduire une localisation fine (coordonnées, distances), nous ajoutons
une variable géographique simple : le **département**.

Le département permet de capter des différences régionales de prix
(tension du marché, attractivité, niveau de vie),
tout en restant une information grossière et facile à interpréter.

Cette étape permet de mesurer l’apport de la localisation
par rapport à un modèle basé uniquement sur les caractéristiques du logement.


In [ ]:
df_model_2 = df_model.copy()

df_model_2["Code departement"] = df_model_2["Code departement"].astype(str) # important pour traiter correctement des codes comme 01, 2A, 2B

In [ ]:
# Création des variables indicatrices pour les départements 
departement_dummies = pd.get_dummies(
    df_model_2["Code departement"],
    prefix="Dep",
    drop_first=True
)
# Pour test : df_model_2.sample(n=5)

In [ ]:
# On refait les mêmes manipulations

df_model_2_clean = df_model_2[["Surface reelle bati", "Appartement", "Nombre pieces principales","Valeur m2","Code departement"]].dropna()
print(df_model.shape[0]-df_model_clean.shape[0])

X_num = df_model_2[["Surface reelle bati", "Appartement", "Nombre pieces principales","Code departement"]]
X_num = pd.concat([X_num, departement_dummies], axis=1)
X_num = X_num.drop("Code departement",axis=1)
# X_num.sample(n=5)

X_num["Surface reelle bati"] = np.log(X_num["Surface reelle bati"])
y_num = np.log(df_model_2_clean["Valeur m2"])

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_num, y_num, test_size=0.2, random_state=42
)

model = LinearRegression()

model.fit(X_train2, y_train2)

print(r2_score(y_test2, model.predict(X_test2)))

L’ajout du département améliore fortement le pouvoir explicatif du modèle ($R^2$ = 26%).
Cela confirme que la localisation explique une part majeure des différences
de prix au mètre carré.

Cette variable capte des effets régionaux (tension du marché, attractivité),
sans recourir à une localisation fine.

### 4.2 Variables départementales issues du scraping

Afin d’enrichir la base DVF avec des informations de contexte géographique,
nous construisons une base de données départementale à partir d’un **scraping de Wikipédia**.

L’objectif est de récupérer, pour chaque département français :
- le code INSEE ;
- le nom du département ;
- la préfecture ;
- la population ;
- la densité de population.

Ces variables permettent de capter des effets macro-géographiques
(niveau d’urbanisation, pression démographique),
sans recourir à une localisation fine.

Nous utilisons Wikipédia comme source car les informations sont publiques et structurées sous forme de tableaux.


In [ ]:
pip install lxml

In [ ]:
import requests
import io
import re

# 1. RÉCUPÉRATION DES DONNÉES

url = "https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais"

# En-tête pour identifier proprement la requête HTTP, puis requête vers Wikipédia
headers = {"User-Agent": "Etudiant_DataScience_Immo/1.0"}
response = requests.get(url, headers=headers)

# Extraction du tableau HTML contenant le mot "Densité"
dfs = pd.read_html(io.StringIO(response.text), match="Densité")
df_dept = dfs[0].copy()


# 2. SÉLECTION ET RENOMMAGE DES COLONNES UTILES

df_final = df_dept.iloc[:, [0, 1, 2, 11, 12]].copy()

df_final.columns = [
    'Code_Insee',
    'Département',
    'Préfecture',
    'Population_Dept',
    'Densité_Dept'
]

# 3. FONCTIONS DE NETTOYAGE

def clean_insee(code):
    # Nettoie et harmonise les codes département (01, 2A, 2B, 971, etc.)
    code = str(code).strip()
    if '2A' in code: return '2A'
    if '2B' in code: return '2B'
    digits = "".join(filter(str.isdigit, code))
    if digits.startswith('97'): return digits[:3]
    if len(digits) == 3: return digits[:2]
    return digits[:2].zfill(2)

def clean_text(text):
    # Nettoie les noms
    if pd.isna(text): return ""
    text = str(text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    return text.strip()

df_final['Code_Insee'] = df_final['Code_Insee'].apply(clean_insee)
df_final['Département'] = df_final['Département'].apply(clean_text)
df_final['Préfecture'] = df_final['Préfecture'].apply(clean_text)

# Suppression des lignes parasites
df_final = df_final[df_final['Code_Insee'] != '']

# Retraitement final pour la population
df_final['Population_Dept'] = (
    pd.to_numeric(
        df_final['Population_Dept'].astype(str)
        .str.replace(r'\D', '', regex=True)
        .str[:-4],
        errors='coerce'
    )
    .fillna(0)
    .astype(int)
)

df_final['Densité_Dept'] = (
    pd.to_numeric(
        df_final['Densité_Dept'].astype(str)
        .str.replace(r'\[.*?\]', '', regex=True)
        .str.replace(r'\D', '', regex=True),    
        errors='coerce'
    )
    .fillna(0)
    .astype(int)
)

# Vérification finale
print(df_final.head())
print(df_final.dtypes)

#### Intégration des variables issues du scraping

Nous intégrons maintenant à la base DVF les variables départementales obtenues par scraping :
- la population du département ;
- la densité de population.

Ces variables permettent de capter des effets macro-géographiques
(niveau d’urbanisation, pression démographique).

In [ ]:
# Copie du jeu de données DVF
df_model3 = df.copy()
df_model3["Code departement"] = df_model3["Code departement"].astype(str)

# Jointure avec les données issues du scraping
df_model3 = df_model3.merge(
    df_final[["Code_Insee", "Population_Dept", "Densité_Dept"]],
    left_on="Code departement",
    right_on="Code_Insee",
    how="left"
)

# Vérification rapide des valeurs manquantes après jointure
df_model3[["Population_Dept", "Densité_Dept"]].isna().mean()

Nous enrichissons la régression en ajoutant les variables départementales
de population et de densité.

Afin de conserver une spécification adaptée aux données immobilières,
nous utilisons une transformation logarithmique pour les variables continues.


In [ ]:
# Encodage du type de bien
df_model3["Appartement"] = (df_model3["Type local"] == "Appartement").astype(int)

# Transformations logarithmiques
df_model3["log_surface"] = np.log(df_model3["Surface reelle bati"])
df_model3["log_population"] = np.log(df_model3["Population_Dept"])
df_model3["log_densite"] = np.log(df_model3["Densité_Dept"])

# Sélection des variables explicatives
X = df_model3[
    ["log_surface", "Appartement", "Nombre pieces principales",
     "log_population", "log_densite"]
]

# Variable cible
y = np.log(df_model3["Valeur m2"])

# Nettoyage des lignes incomplètes
df_model3_clean = pd.concat([X, y], axis=1).dropna()
X = df_model3_clean[X.columns]
y = df_model3_clean[y.name]


Nous estimons une nouvelle régression linéaire incluant les variables
issues du scraping afin d’évaluer leur contribution explicative.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entraînement du modèle
model = LinearRegression()
model.fit(X_train, y_train)

# Évaluation du modèle
r2_scrap = r2_score(y_test, model.predict(X_test))
r2_scrap


Le modèle utilisant les variables issues du scraping présente un R² inférieur à celui utilisant des indicatrices départementales.
Cela s’explique par le fait que les dummies départementales captent l’ensemble des effets spécifiques à chaque département, tandis que la population et la densité n’en captent qu’une partie structurelle.
En contrepartie, le modèle avec scraping est plus interprétable économiquement et permet de comprendre quels mécanismes sous-jacents expliquent les différences de prix.

In [ ]:
# Affichage des coefficients estimés
pd.Series(
    model.coef_,
    index=X.columns
)


La régression estimée est de type log–log, ce qui permet d’interpréter les coefficients
comme des effets proportionnels, toutes choses égales par ailleurs.

Le coefficient associé à la surface réelle bâtie est égal à **−0,44**.
Cela signifie qu’une augmentation de 10 % de la surface d’un logement
est associée à une baisse d’environ **4,4 %** du prix au mètre carré.
Ce résultat met en évidence un effet taille marqué,
traduisant des rendements décroissants de la surface,
phénomène classique sur le marché immobilier résidentiel.

Le coefficient de la variable indiquant un appartement est égal à **0,22**.
À caractéristiques identiques, un appartement présente donc un prix au mètre carré
environ **22 % plus élevé** qu’une maison.
Cet effet reflète des différences structurelles de marché,
les appartements étant plus fréquemment situés dans des zones urbaines denses et tendues.

Le nombre de pièces principales a un coefficient égal à **0,04**.
À surface donnée, une pièce supplémentaire est associée
à une hausse d’environ **4 %** du prix au mètre carré,
ce qui suggère qu’un logement mieux découpé ou plus fonctionnel
est davantage valorisé.

Le coefficient associé au log population départementale est égal à **0,12**.
Une augmentation de 10 % de la population d’un département
est donc associée à une hausse d’environ **1,2 %** du prix au mètre carré.
Cette variable capte un effet de pression démographique et d’attractivité territoriale.

Enfin, le coefficient de la densité de population est égal à **0,2**.
Une augmentation de 10 % de la densité départementale
est associée à une hausse d’environ **2 %** du prix au mètre carré,
confirmant le rôle de l’urbanisation et de la rareté du foncier
dans la formation des prix immobiliers.

## 5. Localisation fine et modèles avancés


### 5.1 Intégration des gares SNCF

Nous intégrons les gares SNCF afin d’introduire une information de localisation
liée à l’accessibilité ferroviaire.

Les données utilisées proviennent du jeu de données officiel des gares SNCF,
au format CSV.
Dans un premier temps, nous construisons une variable simple indiquant
si une gare est présente dans une commune, avec une représentation visuelle.

In [ ]:
pip install contextily

In [ ]:
import json
import os
import geopandas as gpd
import contextily as ctx

In [ ]:
fichier_json = 'gares-de-voyageurs.geojson'

with open(fichier_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

# On cible la clé 'features' qui contient la liste des gares pour manipuler avec pd
gares = pd.json_normalize(data['features'])
gares = gares.dropna(subset=['geometry.coordinates'])

# Les colonnes qui s'appellent par ex properties.nom, geometry.coordinates sont simplifiées
gares.columns = [c.replace('properties.', '').replace('geometry.', '') for c in gares.columns]

coordinates = pd.DataFrame(gares['coordinates'].tolist(), index=gares.index)
gares['longitude'] = coordinates[0]
gares['latitude'] = coordinates[1]

colonnes_utiles = [
    'nom', 
    'codeinsee', 
    'latitude', 
    'longitude', 
    'segment_drg'
    ]

colonnes_finales = [c for c in colonnes_utiles if c in gares.columns]
gares_clean = gares[colonnes_finales]

print(f"{len(gares_clean)} gares récupérées.")
print(gares_clean.head())

gares_clean.to_csv("gares_sncf_final.csv", index=False)
df_gares_clean = pd.read_csv("gares_sncf_final.csv")

In [ ]:
# Liste des codes communes disposant d'au moins une gare SNCF
communes_avec_gare = df_gares["codeinsee"].astype(str).unique()

# Vérification rapide
len(communes_avec_gare)

In [ ]:
# Représentation visuelle
gdf_gares = gpd.GeoDataFrame(
    df_gares_clean,
    geometry=gpd.points_from_xy(df_gares_clean['longitude'], df_gares_clean['latitude']),
    crs="EPSG:4326"  # Système de coordonnées WGS84
)

fig, ax = plt.subplots(figsize=(15, 15))

gdf_gares.plot(ax=ax, color='red', markersize=5, label='Gares SNCF')
ctx.add_basemap(ax, crs=gdf_gares.crs, source=ctx.providers.OpenStreetMap.Mapnik) # fond de carte OpenStreetMap
plt.title("Carte des gares SNCF en France")

plt.show()

### 5.2 Construction d’une variable d’accessibilité ferroviaire

En l’absence de coordonnées géographiques précises pour les logements,
nous utilisons l’échelle communale pour mesurer l’accessibilité ferroviaire.

Nous construisons une variable binaire indiquant
si la commune du logement dispose d’au moins une gare SNCF.

Avant de pouvoir intégrer les gares SNCF à la base DVF, il est nécessaire
d’harmoniser les identifiants géographiques.

Dans la base DVF, la variable *Code commune* correspond uniquement
au code communal à l’intérieur du département,
tandis que la base des gares SNCF utilise le **code INSEE complet**
(association du code département et du code commune).

Nous reconstruisons donc, côté DVF, un **code commune au format INSEE à 5 chiffres**,
en combinant le code département et le code commune,
afin de rendre les deux bases homogènes et permettre une jointure correcte.

In [ ]:
# Harmonisation du code département (2 chiffres, avec zéros)
df_model3["Code_Insee"] = (
    df_model3["Code_Insee"]
    .astype(str)
    .str.zfill(2)
)

# Harmonisation du code commune DVF (3 chiffres, avec zéros)
df_model3["Code commune"] = (
    df_model3["Code commune"]
    .astype(str)
    .str.zfill(3)
)

df_model3["Code_INSEE"] = df_model3["Code departement"] + df_model3["Code commune"]
df_model3["Code_INSEE"].head()

In [47]:
df_model3["Code_INSEE"] = df_model3["Code_INSEE"].astype(str)

# Création de la variable binaire d’accessibilité ferroviaire
df_model3["gare_commune"] = (
    df_model3["Code_INSEE"]
    .isin(communes_avec_gare)
    .astype(int)
)

# Vérification de la distribution
df_model3["gare_commune"].value_counts(normalize=True)

gare_commune
0   0.59
1   0.41
Name: proportion, dtype: float64

### 5.3 Intégration de l’accessibilité ferroviaire dans le modèle

Nous ajoutons la variable `gare_commune` au modèle log–log afin de mesurer
l’apport marginal de l’accessibilité ferroviaire,
toutes choses égales par ailleurs.

In [49]:
df_model3.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local', 'Surface reelle bati', 'Nombre pieces principales', 'Nature culture', 'Nature culture speciale', 'Surface terrain', 'Valeur m2', 'Code_Insee', 'Population_Dept', 'Densité_Dept', 'Appartement', 'log_surface', 'log_population', 'log_densite', 'Code_INSEE', 'gare_commune'], dtype='object')

In [50]:
# Encodage du type de bien
df_model3["Appartement"] = (df_model3["Type local"] == "Appartement").astype(int)

# Transformations logarithmiques
df_model3["log_surface"] = np.log(df_model3["Surface reelle bati"])
df_model3["log_population"] = np.log(df_model3["Population_Dept"])
df_model3["log_densite"] = np.log(df_model3["Densité_Dept"])

# Variables explicatives (ajout de gare_commune)
X = df_model3[
    [
        "log_surface",
        "Appartement",
        "Nombre pieces principales",
        "log_population",
        "log_densite",
        "gare_commune"
    ]
]

y = np.log(df_model3["Valeur m2"])

df_model3_clean = pd.concat([X, y], axis=1).dropna()
X = df_model3_clean[X.columns]
y = df_model3_clean[y.name]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Évaluation
print(r2_score(y_test, model.predict(X_test)))


0.19335618244353414


In [53]:
df_model3

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Valeur m2,Code_Insee,Population_Dept,Densité_Dept,Appartement,log_surface,log_population,log_densite,Code_INSEE,gare_commune
0,000001,2025-01-07,Vente,468000.00,454,NaN,RUE,0090,DE LA REPUBLIQUE,1550,FARGES,01,158,NaN,B,910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,111.00,5.00,S,NaN,133.00,4216.22,01,679344,1179,0,4.71,13.43,7.07,01158,0
1,000001,2025-01-06,Vente,180000.00,66,NaN,RUE,0050,DE LA FRUITIERE,1200,MONTANGES,01,257,NaN,AC,341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,Appartement,NaN,87.00,4.00,S,NaN,76.00,2068.97,01,679344,1179,1,4.47,13.43,7.07,01257,0
2,000001,2025-01-06,Vente,490000.00,459,B,CHE,0356,DU PETIT CORGENON,1310,BUELLAS,01,065,NaN,B,1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,190.00,6.00,S,NaN,500.00,2578.95,01,679344,1179,0,5.25,13.43,7.07,01065,0
3,000001,2025-01-06,Vente,490000.00,459,B,CHE,0356,DU PETIT CORGENON,1310,BUELLAS,01,065,NaN,B,1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,190.00,6.00,AG,NaN,2452.00,2578.95,01,679344,1179,0,5.25,13.43,7.07,01065,0
4,000001,2025-01-09,Vente,183000.00,250,NaN,AV,0041,DE BELLE FERME,1170,GEX,01,173,NaN,AH,194,NaN,218,"35,74",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,35.00,2.00,NaN,NaN,NaN,5228.57,01,679344,1179,1,3.56,13.43,7.07,01173,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403460,000001,2025-06-23,Vente,835000.00,167,NaN,RUE,8699,SAINT MAUR,75011,PARIS 11,75,111,NaN,AC,71,NaN,207,NaN,211,"91,08",NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,70.00,4.00,NaN,NaN,NaN,11928.57,75,2103778,199599,1,4.25,14.56,12.20,75111,0
403461,000001,2025-06-25,Vente,375000.00,4,B,RUE,3479,FABRE D EGLANTINE,75012,PARIS 12,75,112,NaN,CM,28,NaN,281,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,33.00,1.00,NaN,NaN,NaN,11363.64,75,2103778,199599,1,3.50,14.56,12.20,75112,1
403462,000001,2025-06-25,Vente,1370000.00,2,NaN,AV,7149,PAUL DOUMER,75016,PARIS 16,75,116,NaN,DS,9,NaN,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,92.00,2.00,NaN,NaN,NaN,14891.30,75,2103778,199599,1,4.52,14.56,12.20,75116,1
403463,000001,2025-06-27,Vente,550000.00,84,NaN,RUE,9679,VERGNIAUD,75013,PARIS 13,75,113,NaN,DK,8,NaN,376,NaN,421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,61.00,3.00,NaN,NaN,NaN,9016.39,75,2103778,199599,1,4.11,14.56,12.20,75113,1


### 5.4 Modèles non linéaires (avec les mêmes variables que la dernière régression)

Nous utilisons une Random Forest afin de capturer automatiquement
les non-linéarités et interactions complexes entre variables.

Ce modèle sert de benchmark prédictif.

In [55]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

print(r2_score(y_test, rf.predict(X_test)))

0.41207750666255016


Le modèle Random Forest présente un coefficient de détermination d’environ **41 %**, supérieur à celui des modèles linéaires testés précédemment.

Cette amélioration s’explique par la capacité de la Random Forest à capturer automatiquement
des **relations non linéaires** et des **interactions complexes** entre les variables explicatives,
sans avoir à les spécifier explicitement.
Contrairement à la régression linéaire, le modèle n’impose pas de forme fonctionnelle
a priori entre les variables et le prix au mètre carré.

Ce résultat met en évidence l’existence de mécanismes non linéaires dans la formation des prix immobiliers,
en particulier dans l’interaction entre les caractéristiques intrinsèques du logement
(surface, nombre de pièces, type de bien) et les variables macro-géographiques
(population et densité départementales).

En contrepartie, la Random Forest est moins interprétable que les modèles linéaires :
elle permet d’améliorer la performance prédictive,
mais au prix d’une lecture économique plus limitée des effets marginaux.
Nous utilisons donc ce modèle comme **benchmark prédictif**,
tandis que les modèles linéaires restent privilégiés pour l’interprétation économique.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(
    max_iter=200,
    learning_rate=0.1,
    random_state=42
)

hgb.fit(X_train, y_train)

print(r2_score(y_test, hgb.predict(X_test)))

0.3739032223783837


In [61]:
pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1
)

lgbm.fit(X_train, y_train)

print(r2_score(y_test, lgbm.predict(X_test)))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001729 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 477
[LightGBM] [Info] Number of data points in the train set: 322772, number of used features: 6
[LightGBM] [Info] Start training from score 7.964514
0.3952168264589836


Les modèles de Gradient Boosting testés, **HistGradientBoosting** et **LightGBM**, affichent des $R^2$ respectifs de **37 %** et **40 %**. Bien que ces performances demeurent légèrement en deçà de celles du Random Forest (41 %), elles confirment la pertinence des approches non linéaires sur la régression classique.

La convergence des scores de ces trois algorithmes autour d'un palier de 40 % témoigne de la robustesse de la modélisation : elle suggère que nous avons capturé une grande partie de l'information explicative disponible dans les variables qu'on a sélectionnées, indépendamment de l'algorithme choisi. 

## 6. Gamification

In [69]:
df_model3

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Valeur m2,Code_Insee,Population_Dept,Densité_Dept,Appartement,log_surface,log_population,log_densite,Code_INSEE,gare_commune
0,000001,2025-01-07,Vente,468000.00,454,NaN,RUE,0090,DE LA REPUBLIQUE,1550,FARGES,01,158,NaN,B,910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,111.00,5.00,S,NaN,133.00,4216.22,01,679344,1179,0,4.71,13.43,7.07,01158,0
1,000001,2025-01-06,Vente,180000.00,66,NaN,RUE,0050,DE LA FRUITIERE,1200,MONTANGES,01,257,NaN,AC,341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,Appartement,NaN,87.00,4.00,S,NaN,76.00,2068.97,01,679344,1179,1,4.47,13.43,7.07,01257,0
2,000001,2025-01-06,Vente,490000.00,459,B,CHE,0356,DU PETIT CORGENON,1310,BUELLAS,01,065,NaN,B,1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,190.00,6.00,S,NaN,500.00,2578.95,01,679344,1179,0,5.25,13.43,7.07,01065,0
3,000001,2025-01-06,Vente,490000.00,459,B,CHE,0356,DU PETIT CORGENON,1310,BUELLAS,01,065,NaN,B,1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,190.00,6.00,AG,NaN,2452.00,2578.95,01,679344,1179,0,5.25,13.43,7.07,01065,0
4,000001,2025-01-09,Vente,183000.00,250,NaN,AV,0041,DE BELLE FERME,1170,GEX,01,173,NaN,AH,194,NaN,218,"35,74",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,35.00,2.00,NaN,NaN,NaN,5228.57,01,679344,1179,1,3.56,13.43,7.07,01173,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403460,000001,2025-06-23,Vente,835000.00,167,NaN,RUE,8699,SAINT MAUR,75011,PARIS 11,75,111,NaN,AC,71,NaN,207,NaN,211,"91,08",NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,70.00,4.00,NaN,NaN,NaN,11928.57,75,2103778,199599,1,4.25,14.56,12.20,75111,0
403461,000001,2025-06-25,Vente,375000.00,4,B,RUE,3479,FABRE D EGLANTINE,75012,PARIS 12,75,112,NaN,CM,28,NaN,281,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,33.00,1.00,NaN,NaN,NaN,11363.64,75,2103778,199599,1,3.50,14.56,12.20,75112,1
403462,000001,2025-06-25,Vente,1370000.00,2,NaN,AV,7149,PAUL DOUMER,75016,PARIS 16,75,116,NaN,DS,9,NaN,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,92.00,2.00,NaN,NaN,NaN,14891.30,75,2103778,199599,1,4.52,14.56,12.20,75116,1
403463,000001,2025-06-27,Vente,550000.00,84,NaN,RUE,9679,VERGNIAUD,75013,PARIS 13,75,113,NaN,DK,8,NaN,376,NaN,421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,61.00,3.00,NaN,NaN,NaN,9016.39,75,2103778,199599,1,4.11,14.56,12.20,75113,1


In [71]:
# 1. On crée un référentiel unique des communes à partir de ton df_model3
# On ne garde qu'une ligne par commune pour éviter les doublons
ref_communes = df_model3[[
    "Commune", 
    "Population_Dept", 
    "Densité_Dept", 
    "gare_commune"
]].drop_duplicates(subset="Commune").set_index("Commune")

# On nettoie un peu les index (noms des villes) pour faciliter la recherche (tout en majuscule)
ref_communes.index = ref_communes.index.str.upper().str.strip()

print("Base de connaissance des villes créée !")
print(f"{len(ref_communes)} villes référencées.")

Base de connaissance des villes créée !
24378 villes référencées.


In [72]:
def estimer_mon_bien(model):
    print("🏠 --- ESTIMATEUR IMMOBILIER IA --- 🏠")
    print("Répondez aux questions pour estimer votre bien.\n")

    # --- 1. COLLECTE DES INFOS UTILISATEUR ---
    
    # A. La Commune
    while True:
        ville_input = input("📍 Dans quelle commune se situe le bien ? : ").upper().strip()
        
        if ville_input in ref_communes.index:
            # BINGO : On a trouvé la ville, on récupère ses infos cachées
            infos_ville = ref_communes.loc[ville_input]
            
            # Gestion des cas où il y a des homonymes (plusieurs villes du même nom)
            # On prend la première par défaut si c'est le cas
            if isinstance(infos_ville, pd.DataFrame):
                infos_ville = infos_ville.iloc[0]
                
            pop_dept = infos_ville['Population_Dept']
            densite_dept = infos_ville['Densité_Dept']
            gare = infos_ville['gare_commune']
            print(f"   ✅ Ville identifiée (Dept Pop: {pop_dept}, Gare: {'Oui' if gare==1 else 'Non'})")
            break
        else:
            print(f"   ❌ Oups, je ne connais pas '{ville_input}' ou elle n'est pas dans mon jeu de données.")
            print("   Essayez une autre ville")

    # B. Le Type de bien
    type_bien = input("🏠 Type de bien (Tapez 'A' pour Appartement, 'M' pour Maison) : ").upper()
    is_appartement = 1 if type_bien == 'A' else 0
    
    # C. La Surface
    surface = float(input("sz📏 Surface réelle (m²) : "))
    
    # D. Le Nombre de pièces
    pieces = int(input("r🚪 Nombre de pièces principales : "))

    # --- 2. CALCUL DES VARIABLES (FEATURE ENGINEERING) ---
    
    # Transformation logarithmique comme lors de l'entraînement
    log_surface = np.log(surface)
    log_pop = np.log(pop_dept)
    log_densite = np.log(densite_dept)
    
    # Création du vecteur X pour la prédiction
    # L'ORDRE DOIT ÊTRE EXACTEMENT LE MÊME QUE LORS DU FIT DU MODÈLE
    # Variables : [log_surface, Appartement, Nombre pieces, log_population, log_densite, gare_commune]
    
    X_input = pd.DataFrame([[
        log_surface,
        is_appartement,
        pieces,
        log_pop,
        log_densite,
        gare
    ]], columns=[
        'log_surface', 
        'Appartement', 
        'Nombre pieces principales', 
        'log_population', 
        'log_densite', 
        'gare_commune'
    ])
    
    # --- 3. PRÉDICTION ---
    prix_m2_estime = model.predict(X_input)[0]
    
    # Retour à l'échelle (exponentielle) car on prédit souvent le log(prix) ou directement le prix
    # SI ton modèle prédit directement le log du prix, décommente la ligne suivante :
    # prix_m2_estime = np.exp(prix_m2_estime)
    
    prix_total = prix_m2_estime * surface
    
    print("\n" + "="*40)
    print(f"💰 RÉSULTAT DE L'ESTIMATION")
    print("="*40)
    print(f"Prix au m² estimé : {prix_m2_estime:,.0f} €/m²".replace(',', ' '))
    print(f"VALEUR TOTALE     : {prix_total:,.0f} €".replace(',', ' '))
    print("="*40)

# Pour lancer le jeu, il suffit d'appeler la fonction avec ton modèle (rf, xgb, ou lgbm)
# Exemple :
# estimer_mon_bien(rf)

In [74]:
estimer_mon_bien(rf)

🏠 --- ESTIMATEUR IMMOBILIER IA --- 🏠
Répondez aux questions pour estimer votre bien.

   ✅ Ville identifiée (Dept Pop: 1690493, Gare: Oui)

💰 RÉSULTAT DE L'ESTIMATION
Prix au m² estimé : 8 €/m²
VALEUR TOTALE     : 1 139 €


## 7. Conclusion et perspectives